In [ ]:
import json, requests, xml
from xml.dom import minidom
# %pip install ipyleaflet
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
   LayersControl, Icon, Marker, basemap_to_tiles, Choropleth,
   MarkerCluster, Heatmap, SearchControl, 
   FullScreenControl)


In [ ]:
def show_features_on_map( query_response ):
  data = query_response.json()
  center = [38.128, 2.588]
  zoom = 3
  m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
  geolayer = GeoJSON(
    name = 'Features',
    data=data,
    style={ 'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.7, 'weight': 1 },
    hover_style={ 'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5 }
  )
  m.add_layer(geolayer)
  m.add_control(FullScreenControl())
  m.add_control(LayersControl(position='topright'))
  d = m.layers[1].data   # contains featurecollection or feature
  key = 'features'
  if key in d:
    if (len(d[key])<1) : return m
    d = d[key][0]  # use first feature of collection
  key = 'bbox'
  if key in d:
    c = d[key]
    m.fit_bounds([[ c[1],c[0] ],[ c[3], c[2] ]])
  return m


# FedEO API Tutorial - PART 1: OpenSearch and API Features  
 This tutorial is part of a series of tutorials covering related subjects:                  

 * FedEO API Tutorial - PART 1: OpenSearch and API Features
 * [FedEO API Tutorial - PART 2: SPARQL](tutorial-2-sparql.ipynb) 
 * [FedEO API Tutorial - PART 3: STAC](tutorial-3-stac.ipynb)


<a name='Overview'></a>     
## Overview  
 The different parts of the API covered in this Notebook are:    
 * [LandingPage](#LandingPage)
 * StacCollection
 * [Collections](#Collections)
 * [Collection](#Collection)
 * [APIDefinition](#APIDefinition)
 * [Features](#Features)
 * [Feature](#Feature)
      - [Collection](#Feature--EO-Collection)
      - [Service](#Feature--EO-Service)
      - [Granule](#Feature--EO-Granule)   
 


<a name='LandingPage'></a>     
## LandingPage  
 The LandingPage of the API is available at the `/` path.  It is available in multiple representations
 according to multiple profiles.  The API supports content negotiation.  The preferred media type can either
 be requested by using the `Accept` request HTTP header of by using the `httpAccept` query parameter. 
      
 ![LandingPage](https://plantuml-server.kkeisuke.dev/png/ZP91Jm8n48Nlyoicwq4D4HFFX02B4Cqg1FRWWSlO7PULRalQRbPZ_EyM4UaQ8FRGf7tVctcTj6ST6LUL4W18IbsNnLkXSW2xAjIQ39KeTBdMYfMRktV9wDcRGEqRe-jOIsrm84ci0YcMPEr0cupD4M1N-HGqUNvbSO20PP4nYi88oN2EtwTEPyPMLqPmjxkJx5BNNXYplfzF7lVtlvOxvTYyKEW1hnQxQup2iWiLHqWMOyFaUEItoZ2UJMLL0kmz2QdCpt5A-PzKMnG_0FrgZM_JYqluz0r1RAQrwRMDkL11YSZdQUXNoN233ED-YZ4vaZgFC8Hcmr7hwI4v3QMLSP8FC01Hs1tRcQvLkzlqDF8TvNsGwvyW3UIu8tZV_reS6XY2VqMFLUQ_wnS0.png)


    
**Example: 2**  
>  Representation: JSON   
> Requirement:  EOVOC-CCAT-IF-031030 (1)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 3**  
>  Representation: JSON-LD (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031030 (2)  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 4**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031030 (2)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 5**  
>  Representation: RDF/XML (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031030 (3)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: application/rdf+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'application/rdf+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 6**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031030 (3)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 7**  
>  Representation: Turtle (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031030 (4)  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: text/turtle'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'text/turtle'})
print(response.text)


**Example: 8**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031030 (4)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 9**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031035 (1)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 10**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031035 (2)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 11**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031035 (3)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


<a name='STAC-Collection'></a>     
## STAC Collection  

    
**Example: 12**  
>  Representation: JSON   
> Requirement:  EOVOC-CCAT-IF-033305 (1)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


>  Representation: JSON   
  Ok  
**Example: 13**  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 14**  
>  Representation: JSON (STAC)   
> Requirement:  (no requirement ?)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/series/eo:platform/ALOS/TropForest' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/series/eo:platform/ALOS/TropForest', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 15**  
>  Representation: JSON (STAC)   
> Requirement:  (no requirement ?)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/series/eo:platform/ALOS' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/series/eo:platform/ALOS', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 16**  
>  Representation: JSON (STAC)   
> Requirement:  (no requirement ?)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/series/eo:platform' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/series/eo:platform', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 17**  
>  Representation: JSON (STAC)   
> Requirement:  (no requirement ?)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/earthtopics' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/earthtopics', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 18**  
>  Representation: JSON (STAC)   
> Requirement:  (no requirement ?)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 19**  
>  Representation: JSON-LD (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-033305 (2)  
 NOT OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 20**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-033305 (2)  
 NOT OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 21**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-033305 (2)  
 NOT TESTED  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 22**  
>  Representation: RDF/XML (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-033305 (3)  
 NOT TESTED  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: application/rdf+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'application/rdf+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 23**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-033305 (3)  
 NOT TESTED  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 24**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-033305 (3)  
 NOT TESTED  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 25**  
>  Representation: Turtle (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-033305 (4)  
 NOT TESTED  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: text/turtle'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'text/turtle'})
print(response.text)


**Example: 26**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-033305 (4)  
 NOT TESTED  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 27**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-033305 (4)  
 NOT TESTED  
> ```bash
> curl -v 'https://eovoc.spacebel.be/concepts/instruments' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/concepts/instruments', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


<a name='Collections'></a>     
## Collections  
 The `/collections` path is the endpoint for obtaining the list of feature collections.  `series`, `datasets` and `services` are the available feature collections.  
 The feature collection `series` contains the EO collections while the `datasets` feature collection contains the EO granules.  
      
 ![Collections](https://plantuml-server.kkeisuke.dev/png/bP91Jm8n48Nlyoicwq6D0HAFXX1a4Q918S31WvVQ7TbAjoNJRbPZ_Ezs9KWL8Rg7JNRUzsRdJTjrNh0l2mq0GcjR5Uf5cGM0Mogp4YmAbBPOMKF6p_oh9WpiUQHUCjigjTeozhIGoqXAjN2kPpaZtWDiEX-2ne_F97yX85SPeLGiDS4nVdwrsrDojcH9dSv6ShcjWZ0aUpCRtsthNvPhuucVHEo1eAfD6PEuxOD94nGEKoRXQH1U9HCUJNHP06mz8s6oiCU9MFoOQetY6s1upePNyvDJU8-T8QCEWPKrxa_E-4yDLelSDphVcbpK0LR2v_NmBRbL4WIGzHdLm7ev8LsaXpr04XUmcTdAD3lx37WORupw_t1C-uFM_UrePwYzXdd9NjCE7r9soMJXtdu0.png)


    
**Example: 28**  
>  Representation: JSON   
> Requirement:  EOVOC-CCAT-IF-031210 (1)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 29**  
>  Representation: JSON-LD (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031210 (2)  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 30**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031210 (2)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 31**  
>  Representation: RDF/XML (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031210 (3)  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: application/rdf+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'application/rdf+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 32**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031210 (3)  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 33**  
>  Representation: Turtle (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031210 (4)  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: text/turtle'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'text/turtle'})
print(response.text)


**Example: 34**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031210 (4)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 35**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031212 (1)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 36**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031212 (2)  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 37**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031212 (3)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


<a name='Collection'></a>     
## Collection  
 The `/collections/{collection-id}` path is the endpoint for obtaining a feature collection description.  `series`, `datasets` and `services` are the available feature collections.  
 The feature collection `series` contains the EO collections while the `datasets` feature collection contains the EO granules.  
      
 ![Collection](https://plantuml-server.kkeisuke.dev/png/VPB1Jy8m6CNl_IiQFAWnWmYU35cG8KI3Oe231o-r-sILhYLjbsa8_xijE6YJwWvDzhxVUtdzqe4sH9cgv0WXmhciI_R5n0eXlMPYGnGfCPNbHWeGPc6-EM3BThtfdLAopYIN2WyveMjlb76YzL2g7DGVG9Fy7pHx_mGQ8L31Si2KASe1dUFzNxy_1oqhHI5Dcv4kP6q74v2FYzdpIJzQ7eG1zK5y3x9JriYuvSU-YQo5YSQP0c9WR8zA0JvxuLLfRIUJhSfjRoQ53ciTMBn5s7vNa_lbnINQnPsXqVCbzCWaB4T144BKxTsrFbpODZxI--hq-f3Yw81DfgCOtlqj6_3pqRZp-ZIDyJt71_2oKeP3ZBrfM7zVYMAhmYIflufRbxCXfd0fdMsmW5tBz-DsBclHJjqzutetBlVYyerTwG14RX_-3m00.png)


    
**Example: 38**  
>  Representation: JSON   
> Requirement:  EOVOC-CCAT-IF-031310 (1)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 39**  
>  Representation: JSON-LD (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031310 (2)  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 40**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031310 (2)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 41**  
>  Representation: RDF/XML (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031310 (3)  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: application/rdf+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'application/rdf+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 42**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031310 (3)  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 43**  
>  Representation: Turtle (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031310 (4)  
  NOT OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: text/turtle'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'text/turtle'})
print(response.text)


**Example: 44**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031310 (4)  
  NOT OK - too many unused namespaces included.  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 45**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031312 (1)  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 46**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031312 (2)  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 47**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031312 (3)  
  NOT OK - duplicate namespace dct, dcterms  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


**Example: 48**  
>  Representation: text/html   
> Requirement:  EOVOC-CCAT-IF-035430  
>  Kind: EO Series  
  NOT OK - wrong media type (no HTML returned).  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series' -H 'Accept: text/html'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series', headers={'Accept': 'text/html'})
print(response.text)


<a name='APIDefinition'></a>     
## APIDefinition  
 The `APIDefinition` is available at the `/api` path.  It is available in multiple representations
 according to multiple profiles.   OpenSearch clients can obtain the OpenSearch Description Document (OSDD) and more 
 sophisticated clients may prefer using the OpenAPI Definition.
      
 ![APIDefinition](https://plantuml-server.kkeisuke.dev/png/ZPDHIyCm58NVyoj2VL2HJV1HHjdMpJ6PRZWV9kpbcjwjSMbIafHDnFzkugZjfAtseT2SyvtSUw1zOq7RF1M44112xLD-u79BYDbncO66b3ALPagYj4lxBf0wxqr57MYjzf4IIjEX0BQhI940OuPAnwWR34Locsd--eQina4I7YDbN3E1v9n-V_LwJsXKhXc6OI6PHEsTC45rlvm_bkS_o5HQr1keCiIfl3YcGJLsBQE0Wg6HHh1uvrwvHdgs47bAICdCGCQknmLiVqrrjD8FGjrpDHa_NroIpoevM4n7kE6IMwxaALj1Iv8GvdlsKnuNNCjvXjB5q95YUUhg3MZfUHaypDezNbaYQ9RG4HgcUUOd2TmeAu-MKQjZqlYG2U2ozhPYeXeFgLRKqNoRs4xuQ_jRdv61JNp6DMJSSy699w3TMErbD_oZW1QgfP96gg6ajckQwlhFGdNzzL76xcVy0W00.png)

   
**Example: 49**  
>  Representation: OSDD   
> Requirement:  None.  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/api' -H 'Accept: application/opensearchdescription+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/api', headers={'Accept': 'application/opensearchdescription+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 50**  
>  Representation: OpenAPI   
> Requirement:  None.  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/api' -H 'Accept: application/openapi+json;version=3.0'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/api', headers={'Accept': 'application/openapi+json;version=3.0'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


<a name='Features'></a>     
## Features  
 The `/collections/{collection}/items` path is the endpoint for performing searches in one of the available feature collections.
 All search parameters defined by the OpenSearch OSDD and/or the OpenAPI definition of the FedEO Catalog API can be used.  
      
 ![Features](https://plantuml-server.kkeisuke.dev/png/XPDTJy8m58Rl-okQUO76096l2xCiV0bgK0XmuGKtTJkmIjUIjiiqXl_kEXaReSgvwjxtEU_fHzPLcaWTHmoP8eo99AAVb6yGKblATqII2FiYsWaEN2_q5mESqerBTk7sf1J9G30XSPyHVrko1emerHSo0Fa7a2V_1qtVFy2t42Ya0M2VIfy1gk3iotNde4GiVV2ymbAXI59h3E9vCNsrE3qjelhRowHi7VAUk0Qv9kN0j6YkfaTWBDqU5Vnq-DMCnH7-HXZZs_73yle6xK_wHa1qB45TwggOvMowE2IKnbKbtOIwwXLHzzXntHtHeUUjU9rgY9IJlRjl7iAOnojOyRpBmKJXzLqwT5ycizieKndMC7ipu94pSTN9qCAspjZvS5ItvpP_nn_9PImr0mlOFWjzN5eJEpc8a5DOjK0alEQPyrhbXbLjMjMMLMrRrGuwgIxm8FsDVW00.png)


**Example: 51**  
>  Representation: GeojSON   
> Requirement:  None.  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items' -H 'Accept: application/geo+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items', headers={'Accept': 'application/geo+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 53**  
>  Representation: GeojSON   
> Requirement:  None.  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: application/geo+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'application/geo+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 55**  
>  Representation: JSON-LD (default)   
> Requirement:  EOVOC-CCAT-IF-035020  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 56**  
>  Representation: JSON-LD (OGC)   
> Requirement:  EOVOC-CCAT-IF-031528  
 OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 57**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-035022  
 NOT OK - empty response is incorrect (envelope missing)  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 58**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-035024  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 59**  
>  Representation: RDF/XML (OGC)   
> Requirement:  EOVOC-CCAT-IF-035026  
  TBD    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: application/rdf+xml;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'application/rdf+xml;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 60**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-035028  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 61**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-035030  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 62**  
>  Representation: Turtle (OGC)   
> Requirement:  EOVOC-CCAT-IF-035032  
  NOT OK - wrong media type    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: text/turtle;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'text/turtle;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
print(response.text)


**Example: 63**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-035034  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


**Example: 64**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-035036  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 65**  
>  Representation: text/html   
> Requirement:  EOVOC-CCAT-IF-035430  
>  Kind: EO Series  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items' -H 'Accept: text/html'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items', headers={'Accept': 'text/html'})
print(response.text)


<a name='Feature'></a>     
## Feature  
 The `/collections/{collection-id}/items/{id}` path allows for obtaining a `Feature` in a specific representation.  
      
 ![Features](https://plantuml-server.kkeisuke.dev/png/ZPDHJy8m58NV-okQ-Q3632883uOio1YeGI4m4nzugLjXawubRPTf2F_TTh0nicfyMRfpltFkxLtM5n9ncIO40808OLaIVyLq3O3On7IBE4fWm98jevZAXVmc62gsLQa-SCuobn76uO2WO5CfkGG9CM0ynFmNe4Z-2vf-VEB0G80e3Z4COXuG32vX_jRhpR5WAG-munGb4R5C5SQOFI-choUzj3nHYVaALJr0LUD2XbOrTabT2o81NOwHn2FrI3c65pEI9ifsCXqhudocd8Cx026y6Nl-rRLgf8wpwKAVzuQ0V_c7tiJplJBjxHWMw0-WnoyllwJ7sFmEgriLfRWJ-FuoEODoggKRdkNfC7iohADtDNG-73MDgUr3xnBqKou9hdETMkIZRyhh5bovUPl7wqZQJhdAU-tU8XbfTtEtEiXs7EujgBPWbO6Da6MquUW5cTIMIMoRn3kJs369NJK0w6CQgl_h1m00.png)

 The representations available depend on the type of resource: i.e. 
 EO Collection, EO Service or EO Granule.


<a name='Feature--EO-Collection'></a>     
### Feature (EO Collection)  
 The representations available for an EO Collection include but are not limited to:
 * ISO19139
 * ISO19139-2
 * OGC 17-084r1
 * DIF-10
 * STAC
 * GeoDCAT-AP
 * Schema.org
 * HTML

>    
**Example: 66**  
>  Representation: ISO19139   
> Requirement:  None.  
>  Kind: EO Collection  
 OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/vnd.iso.19139+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/vnd.iso.19139+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


>    
**Example: 67**  
>  Representation: ISO19139-2   
> Requirement:  None  
>  Kind: EO Collection  
 OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/vnd.iso.19139-2+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/vnd.iso.19139-2+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


>    
**Example: 68**  
>  Representation: DIF-10   
> Requirement:  None  
>  Kind: EO Collection  
 OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/dif10+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/dif10+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 69**  
>  Representation: GeoJSON (default=OGC 17-069r3)   
> Requirement:  ERGO-CCAT-IF-031514  
>  Kind: EO Collection  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/geo+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/geo+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 71**  
>  Representation: GeoJSON (OGC 17-069r3)   
> Requirement:  ERGO-CCAT-IF-031514  
>  Kind: EO Collection  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/geo+json;profile="http://www.opengis.net/spec/ogcapi-features-1/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/geo+json;profile="http://www.opengis.net/spec/ogcapi-features-1/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 73**  
>  Representation: GeoJSON (OGC 17-084r1)   
> Requirement:  ERGO-CCAT-IF-031512  
>  Kind: EO Collection  
 NOT OK - does return OGC 17-069r3 instead.  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/geo+json;profile="http://www.opengis.net/spec/eoc-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/geo+json;profile="http://www.opengis.net/spec/eoc-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 75**  
>  Representation: GeoJSON (OGC 17-084r1)   
> Requirement:  ERGO-CCAT-IF-031512/28  
>  Kind: EO Collection  
 OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/geo+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/geo+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 77**  
>  Representation: GeoJSON (OGC 17-084r1/OGC 14-055r2)   
> Requirement:  ERGO-CCAT-IF-???  
>  Kind: EO Collection  
 NOT OK - Future work  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/geo+json;profile="http://www.opengis.net/spec/owc-geojson/1.0/req/core"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/geo+json;profile="http://www.opengis.net/spec/owc-geojson/1.0/req/core"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 79**  
>  Representation: GeoJSON (OGC 17-084r1/OWC)   
> Requirement:  ERGO-CCAT-IF-031512  
>  Kind: EO Collection  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest?mode=owc' -H 'Accept: application/geo+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest?mode=owc', headers={'Accept': 'application/geo+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 81**  
>  Representation: GeoJSON (STAC)   
> Requirement:  ERGO-CCAT-IF-031516 ?  
>  Kind: EO Collection  
 NOT OK: should return an HTTP error as combination is not possible for collection (should be JSON).  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/geo+json;profile="https://stacspec.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/geo+json;profile="https://stacspec.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 83**  
>  Representation: JSON (STAC)   
> Requirement:  ERGO-CCAT-IF-031526   
>  Kind: EO Collection  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/json;profile="https://stacspec.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/json;profile="https://stacspec.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 84**  
>  Representation: JSON (default)   
> Requirement:  ERGO-CCAT-IF-031526   
>  Kind: EO Collection  
 NOT OK: returns GeoJSON instead of JSON  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 85**  
>  Representation: JSON-LD (default=OGC??)   
> Requirement:  EOVOC-CCAT-IF-031528  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 86**  
>  Representation: JSON-LD (OGC)   
> Requirement:  EOVOC-CCAT-IF-031528  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 87**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031530  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 88**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031532  
>  Kind: EO Collection  
  TBD   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 89**  
>  Representation: RDF/XML (default=OGC??)   
> Requirement:  EOVOC-CCAT-IF-031534  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/rdf+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/rdf+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 90**  
>  Representation: RDF/XML (OGC)   
> Requirement:  EOVOC-CCAT-IF-031534  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/rdf+xml;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/rdf+xml;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 91**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031536  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 92**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031538  
>  Kind: EO Collection  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 93**  
>  Representation: Turtle (default=OGC??)   
> Requirement:  EOVOC-CCAT-IF-031540  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: text/turtle'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'text/turtle'})
print(response.text)


**Example: 94**  
>  Representation: Turtle (OGC)   
> Requirement:  EOVOC-CCAT-IF-031540  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: text/turtle;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'text/turtle;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
print(response.text)


**Example: 95**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031542  
>  Kind: EO Collection  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


**Example: 96**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031544  
>  Kind: EO Collection  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 97**  
>  Representation: text/html   
> Requirement:  EOVOC-CCAT-IF-0335410  
>  Kind: EO Collection  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/series/items/TropForest' -H 'Accept: text/html'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/series/items/TropForest', headers={'Accept': 'text/html'})
print(response.text)


<a name='Feature--EO-Service'></a>     
### Feature (EO Service)  
 The representations available for an EO Service include but are not limited to:
 * ISO19139
 * OGC 19-020r1
 * GeoDCAT-AP
 * Schema.org
 * HTML
    
   
   
**Example: 98**  
>  Representation: GeoJSON (default=OGC 17-069r3)   
> Requirement:  ERGO-CCAT-IF-031514  
>  Kind: EO Service  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/geo+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/geo+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 100**  
>  Representation: GeoJSON (OGC 17-069r3)   
> Requirement:  ERGO-CCAT-IF-031514  
>  Kind: EO Service  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/geo+json;profile="http://www.opengis.net/spec/ogcapi-features-1/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/geo+json;profile="http://www.opengis.net/spec/ogcapi-features-1/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 102**  
>  Representation: GeoJSON (OGC 17-084r1)   
> Requirement:  ERGO-CCAT-IF-031512  
>  Kind: EO Service  
 NOT OK - does return OGC 17-069r3 instead.  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/geo+json;profile="http://www.opengis.net/spec/eopad-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/geo+json;profile="http://www.opengis.net/spec/eopad-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 104**  
>  Representation: GeoJSON (OGC 17-084r1)   
> Requirement:  ERGO-CCAT-IF-031512  
>  Kind: EO Service  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman?mode=owc' -H 'Accept: application/geo+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman?mode=owc', headers={'Accept': 'application/geo+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 106**  
>  Representation: JSON-LD (default=schema.org??)   
> Requirement:  EOVOC-CCAT-IF-031528  
>  Kind: EO Service  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 107**  
>  Representation: JSON-LD (OGC)   
> Requirement:  EOVOC-CCAT-IF-031528  
>  Kind: EO Service  
 NOT OK - wrong mediatype (GeoJSON)  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 108**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031530  
>  Kind: EO Service  
 NOK:  adms:Identifier should have an uppercase when used as @type.  Wrong vcard namespace.  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 109**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031532  
>  Kind: EO Service  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 110**  
>  Representation: RDF/XML (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031534  
>  Kind: EO Service  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/rdf+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/rdf+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 111**  
>  Representation: RDF/XML (OGC)   
> Requirement:  EOVOC-CCAT-IF-031534  
>  Kind: EO Service  
 NOT OK - wrong media type  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/rdf+xml;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/rdf+xml;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 112**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031536  
>  Kind: EO Service  
 NOT OK: missing namespaces  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 113**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031538  
>  Kind: EO Service  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 114**  
>  Representation: Turtle (default=OGC??)   
> Requirement:  EOVOC-CCAT-IF-031540  
>  Kind: EO Service  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: text/turtle'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'text/turtle'})
print(response.text)


**Example: 115**  
>  Representation: Turtle (OGC)   
> Requirement:  EOVOC-CCAT-IF-031540  
>  Kind: EO Service  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: text/turtle;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'text/turtle;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
print(response.text)


**Example: 116**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031542  
>  Kind: EO Service  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


**Example: 117**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031544  
>  Kind: EO Service  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 118**  
>  Representation: text/html   
> Requirement:  EOVOC-CCAT-IF-0335410  
>  Kind: EO Service  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: text/html'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'text/html'})
print(response.text)


<a name='Feature--EO-Granule'></a>     
### Feature (EO Granule)  
 The representations available for an EO Granule include but are not limited to:
 * OGC 10-157r4
 * OGC 10-157r3
 * OGC 17-003r2
 * OGC 17-069r3
 * STAC Item
 * GeoDCAT-AP
 * Schema.org
 * HTML
    
   
   
**Example: 119**  
>  Representation: GeoJSON (default=OGC 17-069r3)   
> Requirement:  ERGO-CCAT-IF-031514  
>  Kind: EO Granule  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/geo+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/geo+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 121**  
>  Representation: GeoJSON (OGC 17-069r3)   
> Requirement:  ERGO-CCAT-IF-031514  
>  Kind: EO Granule  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/geo+json;profile="http://www.opengis.net/spec/ogcapi-features-1/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/geo+json;profile="http://www.opengis.net/spec/ogcapi-features-1/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 123**  
>  Representation: GeoJSON (OGC 17-003r2)   
> Requirement:  ERGO-CCAT-IF-031512  
>  Kind: EO Granule  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/geo+json;profile="http://www.opengis.net/spec/eo-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/geo+json;profile="http://www.opengis.net/spec/eo-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 125**  
>  Representation: GeoJSON (OGC 17-003r2)   
> Requirement:  ERGO-CCAT-IF-031512  
>  Kind: EO Granule  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001?mode=owc'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001?mode=owc', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 126**  
>  Representation: GeoJSON (STAC)   
> Requirement:  ERGO-CCAT-IF-031516  
>  Kind: EO Granule  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/geo+json;profile="https://stacspec.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/geo+json;profile="https://stacspec.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 128**  
>  Representation: Atom   
> Requirement:  ERGO-CCAT-IF-031518  
>  Kind: EO Granule  
 NOT OK: Atom link alternate ends with ?nullnull  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/atom+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/atom+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 129**  
>  Representation: JSON-LD (default)   
> Requirement:  EOVOC-CCAT-IF-031528  
>  Kind: EO Granule  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 130**  
>  Representation: JSON-LD (OGC)   
> Requirement:  EOVOC-CCAT-IF-031528  
>  Kind: EO Granule  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 131**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031530  
>  Kind: EO Granule  
 NOK:  platform and instrument id are empty in prov:wasGeneratedby.  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 132**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031532  
>  Kind: EO Granule  
  OK   
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 133**  
>  Representation: RDF/XML (default)   
> Requirement:  EOVOC-CCAT-IF-031534  
>  Kind: EO Granule  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/rdf+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/rdf+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 134**  
>  Representation: RDF/XML (OGC)   
> Requirement:  EOVOC-CCAT-IF-031534  
>  Kind: EO Granule  
 NOT OK - wrong media type  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/rdf+xml;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/rdf+xml;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 135**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031536  
>  Kind: EO Granule  
 NOT OK: HTTP error 415  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 136**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031538  
>  Kind: EO Granule  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 137**  
>  Representation: Turtle (default=schema.org)   
> Requirement:  EOVOC-CCAT-IF-031540  
>  Kind: EO Granule  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: text/turtle'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'text/turtle'})
print(response.text)


**Example: 138**  
>  Representation: Turtle (OGC)   
> Requirement:  EOVOC-CCAT-IF-031540  
>  Kind: EO Granule  
 NOT OK: wrong media type  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: text/turtle;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'text/turtle;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
print(response.text)


**Example: 139**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-031542  
>  Kind: EO Granule  
 NOT OK: HTTP 415 error  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


**Example: 140**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-031544  
>  Kind: EO Granule  
  OK    
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 141**  
>  Representation: text/html   
> Requirement:  EOVOC-CCAT-IF-0335410  
>  Kind: EO Granule  
  OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001' -H 'Accept: text/html'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/PR1_OPER_CHR_MO2_1P_20070804T103600_N51-024_E004-004_0001', headers={'Accept': 'text/html'})
print(response.text)


#  §5.2.9 IF-CAT-API-Search (STAC)
        <<Resource>> Search
**Example: 142**  
>  Representation: GeoJSON (default)   
> Requirement:  EOVOC-CCAT-IF-033510 (1)  
 NOT OK - empty list   
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: application/geo+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'application/geo+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 144**  
>  Representation: GeoJSON (default)   
> Requirement:  EOVOC-CCAT-IF-033510 (1)  
 NOT OK - empty list  
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 145**  
>  Representation: GeoJSON (httpAccept)   
> Requirement:  EOVOC-CCAT-IF-033510 (1)  
 NOT OK - empty list  
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest?httpAccept=application/geo%2Bjson'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest?httpAccept=application/geo%2Bjson', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)


**Example: 147**  
>  Representation: JSON-LD (default??)   
> Requirement:  EOVOC-CCAT-IF-033510 (2)  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: application/ld+json'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'application/ld+json'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 148**  
>  Representation: JSON-LD (OGC)   
> Requirement:  EOVOC-CCAT-IF-033510 (2) ???  
 NOT TESTED   
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'application/ld+json;profile="http://www.opengis.net/spec/os-geojson/1.0"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 149**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  EOVOC-CCAT-IF-033510 (2)  
  NOT TESTED    
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 150**  
>  Representation: RDF/XML (schema.org)   
> Requirement:  EOVOC-CCAT-IF-033510 (3)  
  NOT TESTED    
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: application/rdf+xml;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'application/rdf+xml;profile="https://schema.org"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 151**  
>  Representation: Turtle (schema.org)   
> Requirement:  EOVOC-CCAT-IF-033510 (4)  
  NOT TESTED    
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: text/turtle;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'text/turtle;profile="https://schema.org"'})
print(response.text)


**Example: 152**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  EOVOC-CCAT-IF-033512 (1)  
 NOT TESTED  
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 153**  
>  Representation: RDF/XML (DCAT)   
> Requirement:  EOVOC-CCAT-IF-033512 (2)  
  NOT TESTED    
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: application/rdf+xml;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'application/rdf+xml;profile="http://data.europa.eu/930/"'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 154**  
>  Representation: Turtle (DCAT)   
> Requirement:  EOVOC-CCAT-IF-033512 (3)  
  NOT TESTED    
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: text/turtle;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'text/turtle;profile="http://data.europa.eu/930/"'})
print(response.text)


**Example: 155**  
>  Representation: HTML   
> Requirement:  (no requirement ?)  
  NOT TESTED    
> ```bash
> curl -v 'https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest' -H 'Accept: text/html'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/search?collections=LANDSAT.ETM.GTC,TropForest', headers={'Accept': 'text/html'})
print(response.text)


#  IF-CAT-API : Requests for IDN integration
        
**Example: 156**  
>  Representation: httpAccept=DIF10   
> Requirement:  DOI for ESA CCI Collection  
 OK   
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request/?httpAccept=application/dif10%2Bxml&parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request/?httpAccept=application/dif10%2Bxml&parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 157**  
>  Representation: DIF10   
> Requirement:  DOI for ESA CCI Collection  
 NOT OK: wrong media type   
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request/?parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36' -H 'Accept: application/dif10+xml'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request/?parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36', headers={'Accept': 'application/dif10+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 158**  
>  Representation: httpAccept=ISO19139-2   
> Requirement:  DOI for ESA CCI Collection  
 OK   
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 159**  
>  Representation: httpAccept=Atom   
> Requirement:  DOI for ESA CCI Collection  
 OK  
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request/?httpAccept=application/atom%2Bxml&parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36&recordSchema=server-choice'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request/?httpAccept=application/atom%2Bxml&parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36&recordSchema=server-choice', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 160**  
>  Representation: httpAccept=DIF10   
> Requirement:  DOI for ESA Collection  
 OK  
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request?httpAccept=application/dif10%2Bxml&uid=GOME_Evl_ClimateProd_TCWV&parentIdentifier=EOP:ESA:EARTH-ONLINE'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request?httpAccept=application/dif10%2Bxml&uid=GOME_Evl_ClimateProd_TCWV&parentIdentifier=EOP:ESA:EARTH-ONLINE', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 161**  
>  Representation: httpAccept=DIF10   
> Requirement:  DIF10 Use_Constraints (ESA)  
 OK  
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request?httpAccept=application/dif10%2Bxml&uid=TropForest&parentIdentifier=EOP:ESA:EARTH-ONLINE'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request?httpAccept=application/dif10%2Bxml&uid=TropForest&parentIdentifier=EOP:ESA:EARTH-ONLINE', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 162**  
>  Representation: httpAccept=DIF10   
> Requirement:  DIF10 Use_Constraints (ESA CCI)  
 OK   
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request/?httpAccept=application/dif10%2Bxml&parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request/?httpAccept=application/dif10%2Bxml&parentIdentifier=EOP:STFC:CEDA-CCI&uid=7e139108035142a9a1ddd96abcdfff36', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 163**  
>  Representation: httpAccept=DIF10   
> Requirement:  DIF10 Use_Constraints (VITO)  
 OK   
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request/?httpAccept=application/dif10%2Bxml&parentIdentifier=EOP:VITO:PDF&uid=urn:ogc:def:EOP:VITO:VGT_S1'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request/?httpAccept=application/dif10%2Bxml&parentIdentifier=EOP:VITO:PDF&uid=urn:ogc:def:EOP:VITO:VGT_S1', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 164**  
>  Representation: httpAccept=DIF10   
> Requirement:  DIF10 Use_Constraints (VITO TERRASCOPE)  
 OK   
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request?httpAccept=application/dif10%2Bxml&uid=urn:eop:VITO:TERRASCOPE_S2_FCOVER_V2&parentIdentifier=EOP:VITO:TERRASCOPE'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request?httpAccept=application/dif10%2Bxml&uid=urn:eop:VITO:TERRASCOPE_S2_FCOVER_V2&parentIdentifier=EOP:VITO:TERRASCOPE', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 165**  
>  Representation: httpAccept=DIF10   
> Requirement:  DIF10 Use_Constraints (DLR)  
 OK/NOT OK - uid for client URL and ISO file are different ?  
> ```bash
> curl -v 'https://geo.spacebel.be/opensearch/request?httpAccept=application/dif10%2Bxml&uid=fa8dc12c-b6c5-4ff4-9781-a39c8775d4fa&parentIdentifier=EOP:DLR:GEOSERVICE'
> ```


In [ ]:
response = requests.get('https://geo.spacebel.be/opensearch/request?httpAccept=application/dif10%2Bxml&uid=fa8dc12c-b6c5-4ff4-9781-a39c8775d4fa&parentIdentifier=EOP:DLR:GEOSERVICE', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


# Service Metadata and Discovery Best Practices     


<a name='Atom-Encoding'></a>     
## Atom Encoding     
**Example: 166**  
>  Representation: Atom   
> Requirement:  Online service   
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr?facetLimit=0' -H 'Accept: application/atom+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr?facetLimit=0', headers={'Accept': 'application/atom+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 167**  
>  Representation: Atom   
> Requirement:  Service endpoint  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube?facetLimit=0' -H 'Accept: application/atom+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube?facetLimit=0', headers={'Accept': 'application/atom+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 168**  
>  Representation: Atom   
> Requirement:  Downloadable tool   
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman?facetLimit=0' -H 'Accept: application/atom+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman?facetLimit=0', headers={'Accept': 'application/atom+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 169**  
>  Representation: Atom   
> Requirement:  Jupyter Notebook  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/coastline-classifier?facetLimit=0' -H 'Accept: application/atom+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/coastline-classifier?facetLimit=0', headers={'Accept': 'application/atom+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


<a name='ISO19139-Encoding'></a>     
## ISO19139 Encoding     
**Example: 170**  
>  Representation: ISO19139   
> Requirement:  Online service   
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr?httpAccept=application/vnd.iso.19139%2Bxml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr?httpAccept=application/vnd.iso.19139%2Bxml', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 171**  
>  Representation: ISO19139   
> Requirement:  Online service   
 NOT OK - wrong media type  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr' -H 'Accept: application/vnd.iso.19139+xml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr', headers={'Accept': 'application/vnd.iso.19139+xml'})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 172**  
>  Representation: ISO19139   
> Requirement:  Service endpoint   
 NOT OK - license info missing in example  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube?httpAccept=application/vnd.iso.19139%2Bxml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube?httpAccept=application/vnd.iso.19139%2Bxml', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 173**  
>  Representation: ISO19139   
> Requirement:  Downloadable tool    
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman?httpAccept=application/vnd.iso.19139%2Bxml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman?httpAccept=application/vnd.iso.19139%2Bxml', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


**Example: 174**  
>  Representation: ISO19139   
> Requirement:  Jupyter Notebook    
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/coastline-classifier?httpAccept=application/vnd.iso.19139%2Bxml'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/coastline-classifier?httpAccept=application/vnd.iso.19139%2Bxml', headers={'Accept': ''})
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ')
print(xmlstr)


<a name='OGC-19-020r1-Encoding'></a>     
## OGC 19-020r1 Encoding     
**Example: 175**  
>  Representation: GeoJSON (OGC-19-020r1)   
> Requirement:  Online service   
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr?mode=owc'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr?mode=owc', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 176**  
>  Representation: GeoJSON (OGC-19-020r1)   
> Requirement:  Service endpoint   
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube?mode=owc'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube?mode=owc', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 177**  
>  Representation: GeoJSON (OGC-19-020r1)   
> Requirement:  Downloadable tool  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman?mode=owc'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman?mode=owc', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 178**  
>  Representation:GeoJSON (OGC-19-020r1)   
> Requirement:  Jupyter Notebook  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/coastline-classifier?mode=owc'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/coastline-classifier?mode=owc', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


<a name='GeoDCAT-AP-Encoding'></a>     
## GeoDCAT-AP Encoding     
**Example: 179**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  Online service   
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 180**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  Service endpoint   
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 181**  
>  Representation: JSON-LD (DCAT)   
> Requirement:  Downloadable tool  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 182**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  Jupyter Notebook  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/coastline-classifier' -H 'Accept: application/ld+json;profile="http://data.europa.eu/930/"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/coastline-classifier', headers={'Accept': 'application/ld+json;profile="http://data.europa.eu/930/"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


<a name='Schema-org-Encoding'></a>     
## Schema.org Encoding     
**Example: 183**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  Online service  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/harmony-netcdf-to-zarr', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 184**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  Service endpoint  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/eo-pdgs-landsat-datacube', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 185**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  Downloadable tool  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/rasdaman' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/rasdaman', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 186**  
>  Representation: JSON-LD (schema.org)   
> Requirement:  Jupyter Notebook  
 TBD  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/services/items/coastline-classifier' -H 'Accept: application/ld+json;profile="https://schema.org"'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/services/items/coastline-classifier', headers={'Accept': 'application/ld+json;profile="https://schema.org"'})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


<a name='UMM-JSON-Encoding'></a>     
## UMM-JSON Encoding     
**Example: 187**  
>  Representation: UMM-JSON (UMM-S)   
> Requirement:  Online service   
 TBD  
> ```bash
> curl -v 'https://cmr.earthdata.nasa.gov/search/services.umm_json?name=PO.DAAC%20harmony-netcdf-to-zarr&pretty=true'
> ```


In [ ]:
response = requests.get('https://cmr.earthdata.nasa.gov/search/services.umm_json?name=PO.DAAC%20harmony-netcdf-to-zarr&pretty=true', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


>  Representation: UMM-JSON (UMM-T)  
 TBD  
**Example: 188**  
> ```bash
> curl -v 'https://cmr.earthdata.nasa.gov/search/tools.umm_json?name=Proba-V%20MEP&pretty=true'
> ```


In [ ]:
response = requests.get('https://cmr.earthdata.nasa.gov/search/tools.umm_json?name=Proba-V%20MEP&pretty=true', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


<a name='HDSA-examples-of-zarr-data-access-encoding'></a>     
## HDSA examples of zarr data access encoding     
**Example: 189**  
>  Representation: GeoJSON   
> Requirement:  granule with zarr data access  
>  Uses 'offerings' to provide the zarr information.  
 OK  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/ASA_WSS_1PNESA20100426_155838_000000602088_00498_42636_0000_'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/ASA_WSS_1PNESA20100426_155838_000000602088_00498_42636_0000_', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


**Example: 190**  
>  Representation: GeoJSON (STAC)   
> Requirement:  granule with zarr data access  
 Provides zarr-consolidated-metadata as asset.  
> ```bash
> curl -v 'https://eovoc.spacebel.be/collections/datasets/items/ASA_WSS_1PNESA20100426_155838_000000602088_00498_42636_0000_?httpAccept=application/geo%2Bjson;profile=https://stacspec.org'
> ```


In [ ]:
response = requests.get('https://eovoc.spacebel.be/collections/datasets/items/ASA_WSS_1PNESA20100426_155838_000000602088_00498_42636_0000_?httpAccept=application/geo%2Bjson;profile=https://stacspec.org', headers={'Accept': ''})
payload = json.loads(response.text)
print(json.dumps(payload,indent=2))


Display the features on the map.


In [ ]:
show_features_on_map(response)
